In [ ]:
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from scipy.spatial.distance import euclidean
from skimage.feature import greycomatrix, greycoprops
import matplotlib.pyplot as plt
from skimage import feature
import cv2 
import mahotas
import numpy as np
from imutils import contours



In [ ]:
def LAB(img):
    LABimg = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    hist = cv2.calcHist([LABimg], [0], None, [256], [0, 256])
    return hist

In [ ]:
from imutils import paths
dataset = "dataset"
query  = "cracked_0063.jpg"

index = {}
# loop over the images
for imagePath in paths.list_images(dataset):
    # load the image, convert it to grayscale, and describe it
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = LAB(image)

    # update the index dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    index[filename] = hist

# load the query image and extract Local Binary Patterns from it
query = cv2.imread(query)
queryFeatures = LAB(query)

# show the query image and initialize the results dictionary
cv2.imshow("Query", query)
results = {}

# loop over the index
for (k, features) in index.items():
    # compute the chi-squared distance between the current features and the query
    # features, then update the dictionary of results
    d = 0.5 * np.sum(((features - queryFeatures) ** 2) / (features + queryFeatures + 1e-10))
    results[k] = d

# sort the results
results = sorted([(v, k) for (k, v) in results.items()])


# loop over the results
for (i, (score, filename)) in enumerate(results):
    # show the result image
    print("#%d. %s: %.4f" % (i + 1, filename, score))
    image = cv2.imread(filename)
    cv2.imshow("Result #{}".format(i + 1), image)
    cv2.waitKey(0)
    

## for shape

## using hue moments method

In [34]:
def hue_moment_descriptor(img):
    img_len,img_width = img.shape[0],img.shape[1]
    centre = img_len//2,img_width//2
    seuil = img[centre[0]-2:centre[0]+2,centre[1]-2:centre[1]+2].mean()                     #img[img_dim[0]//2,img_dim[1]//2]
    couleur = 255
    options = [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC,cv2.THRESH_TOZERO,cv2.THRESH_TOZERO_INV]
    result = cv2.threshold(img, int(seuil), couleur, options[0])[1]
    # Calculate Moments 
    moments = cv2.moments(result) 
    # Calculate Hu Moments 
    huMoments = cv2.HuMoments(moments)
    return huMoments.reshape((7,))    


In [35]:
from imutils import paths
dataset = "cracked_train"
query = "test.jpg"

index = {}
# loop over the images
for imagePath in paths.list_images(dataset):
    # load the image, convert it to grayscale, and describe it
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = hue_moment_descriptor(gray)

    # update the index dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    index[filename] = hist

# load the query image and extract Local Binary Patterns from it
query = cv2.imread(query)
queryFeatures = hue_moment_descriptor(cv2.cvtColor(query, cv2.COLOR_BGR2GRAY))

# show the query image and initialize the results dictionary
cv2.imshow("Query", query)
results = {}

# loop over the index
for (k, features) in index.items():
    # compute the chi-squared distance between the current features and the query
    # features, then update the dictionary of results
    d = euclidean(features,queryFeatures)
    #d = 0.5 * np.sum(((features - queryFeatures) ** 2) / (features + queryFeatures + 1e-10))
    results[k] = d

# sort the results
results = sorted([(v, k) for (k, v) in results.items()])[:20]


# loop over the results
for (i, (score, filename)) in enumerate(results):
    # show the result image
    print("#%d. %s: %.4f" % (i + 1, filename, score))
    image = cv2.imread(filename)
    cv2.imshow("Result #{}".format(i + 1), image)
    cv2.waitKey(0)
    

#1. cracked_train\cracked_0149.jpg: 0.0000
#2. cracked_train\cracked_0110.jpg: 0.0000
#3. cracked_train\cracked_0165.jpg: 0.0000
#4. cracked_train\cracked_0121.jpg: 0.0000
#5. cracked_train\cracked_0050.jpg: 0.0001
#6. cracked_train\cracked_0049.jpg: 0.0001
#7. cracked_train\cracked_0125.jpg: 0.0001
#8. cracked_train\cracked_0080.jpg: 0.0001
#9. cracked_train\cracked_0088.jpg: 0.0001
#10. cracked_train\cracked_0114.jpg: 0.0001
#11. cracked_train\cracked_0135.jpg: 0.0001
#12. cracked_train\cracked_0069.jpg: 0.0001
#13. cracked_train\cracked_0119.jpg: 0.0001
#14. cracked_train\cracked_0101.jpg: 0.0002
#15. cracked_train\cracked_0158.jpg: 0.0002
#16. cracked_train\cracked_0090.jpg: 0.0002
#17. cracked_train\cracked_0066.jpg: 0.0002
#18. cracked_train\cracked_0144.jpg: 0.0002
#19. cracked_train\cracked_0105.jpg: 0.0002
#20. cracked_train\cracked_0082.jpg: 0.0002


## using hog method

In [3]:
def hog_descriptor(img):
    resized_img1 = resize(img, (128*4, 64*4))
    fd1= hog(resized_img1, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=False, channel_axis=-1)
    return fd1

In [ ]:
from imutils import paths
dataset = "cracked_train"
query = "braided_0115.jpg"

index = {}
# loop over the images
for imagePath in paths.list_images(dataset):
    # load the image, convert it to grayscale, and describe it
    image = cv2.imread(imagePath)
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = hog_descriptor(image)

    # update the index dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    index[filename] = hist

# load the query image and extract Local Binary Patterns from it
query = cv2.imread(query)
queryFeatures = hog_descriptor(query)

# show the query image and initialize the results dictionary
cv2.imshow("Query", query)
results = {}

# loop over the index
for (k, features) in index.items():
    # compute the chi-squared distance between the current features and the query
    # features, then update the dictionary of results
    d = euclidean(features,queryFeatures)
    #d = 0.5 * np.sum(((features - queryFeatures) ** 2) / (features + queryFeatures + 1e-10))
    results[k] = d

# sort the results
results = sorted([(v, k) for (k, v) in results.items()])[:20]


# loop over the results
for (i, (score, filename)) in enumerate(results):
    # show the result image
    print("#%d. %s: %.4f" % (i + 1, filename, score))
    image = cv2.imread(filename)
    cv2.imshow("Result #{}".format(i + 1), image)
    cv2.waitKey(0)
    

#1. cracked_train\cracked_0066.jpg: 31.9830


## using zernike moments

In [22]:
def Zernike_moments(im):
    #preprocessing
    img = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    img_len,img_width = img.shape[0],img.shape[1]
    centre = img_len//2,img_width//2
    seuil = img[centre[0]-2:centre[0]+2,centre[1]-2:centre[1]+2].mean()                     #img[img_dim[0]//2,img_dim[1]//2]
    couleur = 255
    options = [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC,cv2.THRESH_TOZERO,cv2.THRESH_TOZERO_INV]
    result = cv2.threshold(img, int(seuil), couleur, options[0])[1]
    thresh = cv2.erode(result, None, iterations=5)
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    (cnts, _) = contours.sort_contours(cnts, method="left-to-right")
    features = mahotas.features.zernike_moments(thresh, cv2.minEnclosingCircle(cnts[0])[1], degree=8)
    
    return features


In [26]:
from imutils import paths
dataset = "cracked_train"
query = "braided_0115.jpg"

index = {}
# loop over the images
for imagePath in paths.list_images(dataset):
    # load the image, convert it to grayscale, and describe it
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    try:
        hist = Zernike_moments(image)
    except:
        continue
    # update the index dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    index[filename] = hist

# load the query image and extract Local Binary Patterns from it
query = cv2.imread(query)
queryFeatures = Zernike_moments(query)

# show the query image and initialize the results dictionary
cv2.imshow("Query", query)
results = {}

# loop over the index
for (k, features) in index.items():
    # compute the chi-squared distance between the current features and the query
    # features, then update the dictionary of results
    d = euclidean(features,queryFeatures)
    #d = 0.5 * np.sum(((features - queryFeatures) ** 2) / (features + queryFeatures + 1e-10))
    results[k] = d

# sort the results
results = sorted([(v, k) for (k, v) in results.items()])[:20]


# loop over the results
for (i, (score, filename)) in enumerate(results):
    # show the result image
    print("#%d. %s: %.4f" % (i + 1, filename, score))
    image = cv2.imread(filename)
    cv2.imshow("Result #{}".format(i + 1), image)
    cv2.waitKey(0)
    

#1. cracked_train\cracked_0056.jpg: 0.3285
#2. cracked_train\cracked_0067.jpg: 0.3439
#3. cracked_train\cracked_0130.jpg: 0.3826
#4. cracked_train\cracked_0113.jpg: 0.3963
#5. cracked_train\cracked_0076.jpg: 0.4041
#6. cracked_train\cracked_0115.jpg: 0.4209
#7. cracked_train\cracked_0138.jpg: 0.4330
#8. cracked_train\cracked_0164.jpg: 0.4451
#9. cracked_train\cracked_0124.jpg: 0.4519
#10. cracked_train\cracked_0163.jpg: 0.4665
#11. cracked_train\cracked_0083.jpg: 0.4677
#12. cracked_train\cracked_0060.jpg: 0.4713
#13. cracked_train\cracked_0109.jpg: 0.4723
#14. cracked_train\cracked_0070.jpg: 0.4801
#15. cracked_train\cracked_0142.jpg: 0.4824
#16. cracked_train\cracked_0134.jpg: 0.4901
#17. cracked_train\cracked_0098.jpg: 0.5026
#18. cracked_train\cracked_0063.jpg: 0.5254
#19. cracked_train\cracked_0151.jpg: 0.5472
#20. cracked_train\cracked_0165.jpg: 0.5796


## for Texture 

### Using LBP method

In [39]:
def lbpdescriptor(image,numPoints=24,radius=8,eps=1e-7):
        # compute the Local Binary Pattern representation of the image, and then
        # use the LBP representation to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, numPoints, radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=range(0, numPoints + 3),range=(0, numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return hist

In [40]:
from imutils import paths
dataset = "dataset"
query  = "braided_0115.jpg"

index = {}
# loop over the images
for imagePath in paths.list_images(dataset):
    # load the image, convert it to grayscale, and describe it
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = lbpdescriptor(gray)

    # update the index dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    index[filename] = hist

# load the query image and extract Local Binary Patterns from it
query = cv2.imread(query)
queryFeatures = lbpdescriptor(cv2.cvtColor(query, cv2.COLOR_BGR2GRAY))

# show the query image and initialize the results dictionary
cv2.imshow("Query", query)
results = {}

# loop over the index
for (k, features) in index.items():
    # compute the chi-squared distance between the current features and the query
    # features, then update the dictionary of results
    d = 0.5 * np.sum(((features - queryFeatures) ** 2) / (features + queryFeatures + 1e-10))
    results[k] = d

# sort the results
results = sorted([(v, k) for (k, v) in results.items()])


# loop over the results
for (i, (score, filename)) in enumerate(results):
    # show the result image
    print("#%d. %s: %.4f" % (i + 1, filename, score))
    image = cv2.imread(filename)
    cv2.imshow("Result #{}".format(i + 1), image)
    cv2.waitKey(0)
    

#1. dataset\braided_0115.jpg: 0.0000
#2. dataset\braided_0154.jpg: 0.0014
#3. dataset\wrinkled_0066.jpg: 0.0018
#4. dataset\frilly_0021.jpg: 0.0020
#5. dataset\braided_0111.jpg: 0.0021
#6. dataset\cracked_0061.jpg: 0.0021
#7. dataset\stratified_0131.jpg: 0.0022
#8. dataset\smeared_0111.jpg: 0.0023
#9. dataset\chequered_0145.jpg: 0.0028
#10. dataset\bumpy_0152.jpg: 0.0030
#11. dataset\waffled_0119.jpg: 0.0031
#12. dataset\smeared_0102.jpg: 0.0031
#13. dataset\freckled_0117.jpg: 0.0032
#14. dataset\braided_0114.jpg: 0.0032
#15. dataset\zigzagged_0128.jpg: 0.0033
#16. dataset\crystalline_0108.jpg: 0.0034
#17. dataset\spiralled_0056.jpg: 0.0037
#18. dataset\smeared_0118.jpg: 0.0038
#19. dataset\pitted_0144.jpg: 0.0041
#20. dataset\braided_0118.jpg: 0.0041
#21. dataset\matted_0055.jpg: 0.0041
#22. dataset\braided_0142.jpg: 0.0041
#23. dataset\pitted_0154.jpg: 0.0042
#24. dataset\potholed_0069.jpg: 0.0045


KeyboardInterrupt: 

### Using Haralik method

In [42]:
def texture_heralik_descriptor(img):
    return mahotas.features.haralick(img).mean(axis=0)

In [43]:
query = cv2.imread("braided_0115.jpg")
texture_heralik_descriptor(query)

array([ 8.36502566e-05,  2.63655635e+03,  6.94511608e-01,  4.46357768e+03,
        5.35131539e-02,  2.46529452e+02,  1.52177544e+04,  8.82300738e+00,
        1.41415154e+01,  1.03187093e-04,  5.75035486e+00, -2.19378200e-01,
        9.72385107e-01])

In [44]:
from imutils import paths
dataset = "cracked_train"
query = "braided_0115.jpg"

index = {}
# loop over the images
for imagePath in paths.list_images(dataset):
    # load the image, convert it to grayscale, and describe it
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = texture_heralik_descriptor(gray)

    # update the index dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    index[filename] = hist

# load the query image and extract Local Binary Patterns from it
query = cv2.imread(query)
queryFeatures = texture_heralik_descriptor(cv2.cvtColor(query, cv2.COLOR_BGR2GRAY))

# show the query image and initialize the results dictionary
cv2.imshow("Query", query)
results = {}

# loop over the index
for (k, features) in index.items():
    # compute the chi-squared distance between the current features and the query
    # features, then update the dictionary of results
    d = euclidean(features,queryFeatures)
    results[k] = d

# sort the results
results = sorted([(v, k) for (k, v) in results.items()])[:20]


# loop over the results
for (i, (score, filename)) in enumerate(results):
    # show the result image
    print("#%d. %s: %.4f" % (i + 1, filename, score))
    image = cv2.imread(filename)
    cv2.imshow("Result #{}".format(i + 1), image)
    cv2.waitKey(0)
    

#1. cracked_train\cracked_0081.jpg: 557.3513
#2. cracked_train\cracked_0061.jpg: 568.6704
#3. cracked_train\cracked_0099.jpg: 1072.8282
#4. cracked_train\cracked_0163.jpg: 1156.6942
#5. cracked_train\cracked_0104.jpg: 1398.0257
#6. cracked_train\cracked_0072.jpg: 1447.0349
#7. cracked_train\cracked_0079.jpg: 1454.3554
#8. cracked_train\cracked_0159.jpg: 1509.9598
#9. cracked_train\cracked_0151.jpg: 1598.3404
#10. cracked_train\cracked_0122.jpg: 1603.5707
#11. cracked_train\cracked_0087.jpg: 1687.1791
#12. cracked_train\cracked_0078.jpg: 1700.1948


KeyboardInterrupt: 